# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [1]:
!pip install beautifulsoup4 requests pandas

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [12]:
# After inspection of the website structure, we define a function to scrape book data


def scrape_books(min_rating, max_price):
    
    rating_map = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
    books_list = []
    
    base_url = "https://books.toscrape.com/"
    catalogue_url = base_url + "catalogue/"
    
    # Get main page to find all categories
    response = requests.get(base_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all categories
    category_list = soup.find('ul', class_='nav nav-list')
    categories = category_list.find('ul').find_all('li')
    
    # Extract category URLs and names
    category_urls = []
    for category in categories:
        link = category.find('a')
        if link:
            category_name = link.text.strip()
            category_url = base_url + link['href']
            category_urls.append((category_name, category_url))
    
    print(f"Found {len(category_urls)} categories to scrape\n")

    # Scrape each category
    for category_name, category_url in category_urls:
        print(f"Scraping category: {category_name}")
        page_num = 1
        
        while True:
            # Pagination
            if page_num == 1:
                url = category_url
            else:
                url = category_url.replace('index.html', f'page-{page_num}.html')
            
            try:
                response = requests.get(url)
                
                # Stop if page doesn't exist
                if response.status_code == 404:
                    break
                
                soup = BeautifulSoup(response.content, 'html.parser')
                books = soup.find_all('article', class_='product_pod')
                
                if not books:
                    break
                
                # Process each book on the page
                for book in books:
                    # Extract title
                    title = book.find('h3').find('a')['title']
                    
                    # Extract price
                    price = float(book.find('p', class_='price_color').text.replace('£', ''))
                    
                    # Extract rating
                    rating_class = book.find('p', class_='star-rating')['class'][1]
                    rating = rating_map[rating_class]
                    
                    # Apply filters
                    if rating >= min_rating and price <= max_price:
                        availability_elem = book.find('p', class_='instock availability')
                        availability = availability_elem.text.strip() if availability_elem else 'N/A'
                        
                        # Get book detail page URL
                        book_link = book.find('h3').find('a')['href']
                        book_link = book_link.replace('../../../', '')
                        
                        # Scrape individual book page for UPC and Description
                        book_detail_url = catalogue_url + book_link
                        detail_response = requests.get(book_detail_url)
                        detail_soup = BeautifulSoup(detail_response.content, 'html.parser')
                        
                        # Extract UPC from product information table
                        upc = None
                        product_table = detail_soup.find('table', class_='table table-striped')
                        if product_table:
                            rows = product_table.find_all('tr')
                            for row in rows:
                                if 'UPC' in row.find('th').text:
                                    upc = row.find('td').text.strip()
                                    break
                        
                        # Extract description
                        description = None
                        description_div = detail_soup.find('div', id='product_description')
                        if description_div:
                            description_p = description_div.find_next_sibling('p')
                            if description_p:
                                description = description_p.text.strip()
                        
                        # Add book to list
                        books_list.append({
                            'UPC': upc,
                            'Title': title,
                            'Price (£)': price,
                            'Rating': rating,
                            'Genre': category_name,
                            'Availability': availability,
                            'Description': description
                        })
                        

                
                print(f"  - Page {page_num}: Found {len(books)} books")
                page_num += 1
                
            except Exception as e:
                print(f"  - Error on page {page_num}: {e}")
                break
    
    print(f"\nTotal books found: {len(books_list)}")
    
    return pd.DataFrame(books_list)


# min_rating=4.0 et and max_price=20.0
books_df = scrape_books(min_rating=4.0, max_price=20.0)


print(books_df.head(10))

# Statistiques
print(f"\nTotal books: {len(books_df)}")
print(f"Average price: £{books_df['Price (£)'].mean():.2f}")
print(f"\nRating distribution:")
print(books_df['Rating'].value_counts().sort_index())
print(f"\nTop 5 genres:")
print(books_df['Genre'].value_counts().head())



Found 50 categories to scrape

Scraping category: Travel
  - Page 1: Found 11 books
Scraping category: Mystery
  - Page 1: Found 20 books
  - Page 2: Found 12 books
Scraping category: Historical Fiction
  - Page 1: Found 20 books
  - Page 2: Found 6 books
Scraping category: Sequential Art
  - Page 1: Found 20 books
  - Page 2: Found 20 books
  - Page 3: Found 20 books
  - Page 4: Found 15 books
Scraping category: Classics
  - Page 1: Found 19 books
Scraping category: Philosophy
  - Page 1: Found 11 books
Scraping category: Romance
  - Page 1: Found 20 books
  - Page 2: Found 15 books
Scraping category: Womens Fiction
  - Page 1: Found 17 books
Scraping category: Fiction
  - Page 1: Found 20 books
  - Page 2: Found 20 books
  - Page 3: Found 20 books
  - Page 4: Found 5 books
Scraping category: Childrens
  - Page 1: Found 20 books
  - Page 2: Found 9 books
Scraping category: Religion
  - Page 1: Found 7 books
Scraping category: Nonfiction
  - Page 1: Found 20 books
  - Page 2: Found 20 